In [1]:
import pandas as pd
from tqdm.notebook import tqdm
import requests
import re

In [2]:
base = 'https://www.archives-resultats-elections.interieur.gouv.fr/telechargements/LG2002/T2Valide/'

In [3]:
departements_text = (
    requests
    .get(base)
    .text
)

departements_text

'<html>\r\n<head><title>Index of /telechargements/LG2002/T2Valide/</title></head>\r\n<body bgcolor="white">\r\n<h1>Index of /telechargements/LG2002/T2Valide/</h1><hr><pre><a href="../">../</a>\r\n<a href="001/">001/</a>                                               04-Apr-2007 13:58                   -\r\n<a href="002/">002/</a>                                               04-Apr-2007 14:00                   -\r\n<a href="003/">003/</a>                                               05-Apr-2007 08:04                   -\r\n<a href="004/">004/</a>                                               05-Apr-2007 08:05                   -\r\n<a href="005/">005/</a>                                               05-Apr-2007 08:05                   -\r\n<a href="006/">006/</a>                                               05-Apr-2007 08:05                   -\r\n<a href="007/">007/</a>                                               05-Apr-2007 08:06                   -\r\n<a href="008/">008/</a>    

In [4]:
departements_list = [ txt[1:-2] for txt in re.findall('\>[A-Za-z0-9]{3}\/\<', departements_text)]

In [5]:
departements = (
    pd.concat([
        pd
        .read_xml(
            f'{base}/{dpt}/circons01.xml',
            xpath='.//Region/Departement',
            encoding='latin1',
            dtype="object"
        )
        for dpt in tqdm(departements_list)
    ])
)

  0%|          | 0/105 [00:00<?, ?it/s]

In [6]:
departements

,CodDpt,CodDpt3Car,LibDpt,Circonscription
0,01,001,AIN,None
0,02,002,AISNE,None
0,03,003,ALLIER,None
0,04,004,ALPES DE HAUTE PROVENCE,None
0,05,005,HAUTES ALPES,None
...,...,...,...,...
0,975,975,SAINT PIERRE ET MIQUELON,None
0,976,976,MAYOTTE,None
0,986,986,WALLIS ET FUTUNA,None
0,987,987,POLYNESIE FRANCAISE,None


In [7]:
circonscriptions = (
    pd.concat([
        
        pd
        .read_xml(
            f'{base}/{dpt}/circons01.xml',
            xpath='.//IdxCirLeg',
            encoding='latin1',
            dtype="object"
        )
        .assign(
            CodDpt3Car = dpt
        )
        
        for dpt in tqdm(departements.CodDpt3Car)
    ])
)

circonscriptions

  0%|          | 0/105 [00:00<?, ?it/s]

,CodCirLg,LibCirLg,NomPageIndex,CodDpt3Car
0,01,1ère circonscription,circons01,001
1,02,2ème circonscription,circons02,001
2,03,3ème circonscription,circons03,001
3,04,4ème circonscription,circons04,001
0,01,1ère circonscription,circons01,002
...,...,...,...,...
0,01,1ère circonscription,circons01,986
0,01,1ère circonscription,circons01,987
1,02,2ème circonscription,circons02,987
0,01,1ère circonscription,circons01,988


In [8]:
t1_resultats = (
    pd.concat([
        pd
        .read_xml(
            f'{base}/{c["CodDpt3Car"]}/circons{c["CodCirLg"]}.xml',
            xpath='.//Tour[./NumTour = "1"]//Candidat',
            encoding='latin1',
            dtype="object"
        )
        .assign(
            CodCirc = c['CodDpt3Car'] + c['CodCirLg'],
            NbVoix = lambda df: df.Voix.str.replace(' ','').astype(int),
            RapportExprimes = lambda df: df.RapVoixExp.str.replace(' ','').str.replace(',','.').astype(float)
        )
        
        for idx, c in tqdm(list(circonscriptions.iterrows()))
    ])
)

t1_resultats

  0%|          | 0/577 [00:00<?, ?it/s]

,Nom,Prenom,Civilite,LibNua,NumDepCand,Voix,RapVoixExp,CodCirc,NbVoix,RapportExprimes
0,GODIN,ANDRE,M.,Socialiste,2,16 734,"33,23",00101,16734,33.23
1,BERTRAND,J.MICHEL,M.,Union pour la majorité présidentielle,3,21 397,"42,49",00101,21397,42.49
2,COTTON,J.PIERRE,M.,Extrême gauche,5,597,"1,19",00101,597,1.19
3,GUENARD-GERBAUD,CAROLE,Mme,Ligue communiste révolutionnaire,7,713,"1,42",00101,713,1.42
4,LE MAOUT,HERVE,M.,Mouvement pour la France,8,803,"1,59",00101,803,1.59
...,...,...,...,...,...,...,...,...,...,...
2,DOUYERE,Eric,M.,Divers gauche,5,728,"2,73",98802,728,2.73
3,GUILLEMARD,justin,M.,Divers droite,10,1 093,"4,10",98802,1093,4.10
4,MAPOU,raphaël,M.,Divers,11,1 172,"4,39",98802,1172,4.39
5,PAITA,gabriel,M.,Divers,14,654,"2,45",98802,654,2.45


In [9]:
t1_resultats.to_csv('lg2002/t1_resultats.csv', index=False)

In [10]:
def get_tour(c, tour):
    results = None
    url = f'{base}{c["CodDpt3Car"]}/circons{c["CodCirLg"]}.xml'
    try:
        results = (
            pd
            .read_xml(
                url,
                xpath='.//Tour[./NumTour = "2"]//Candidat',
                encoding='latin1',
                dtype="object"
            )
            .assign(
                CodCirc = c['CodDpt3Car'] + c['CodCirLg'],
                NbVoix = lambda df: df.Voix.str.replace(' ','').astype(int),
                RapportExprimes = lambda df: df.RapVoixExp.str.replace(' ','').str.replace(',','.').astype(float)
            )
        )
    except:
        print('error', url)
    
    return results

t2_resultats = (
    pd.concat([ get_tour(c, 2) for idx, c in tqdm(list(circonscriptions.iterrows()))])
)

t2_resultats

  0%|          | 0/577 [00:00<?, ?it/s]

error https://www.archives-resultats-elections.interieur.gouv.fr/telechargements/LG2002/T2Valide/006/circons07.xml
error https://www.archives-resultats-elections.interieur.gouv.fr/telechargements/LG2002/T2Valide/008/circons03.xml
error https://www.archives-resultats-elections.interieur.gouv.fr/telechargements/LG2002/T2Valide/013/circons02.xml
error https://www.archives-resultats-elections.interieur.gouv.fr/telechargements/LG2002/T2Valide/014/circons04.xml
error https://www.archives-resultats-elections.interieur.gouv.fr/telechargements/LG2002/T2Valide/015/circons02.xml
error https://www.archives-resultats-elections.interieur.gouv.fr/telechargements/LG2002/T2Valide/021/circons04.xml
error https://www.archives-resultats-elections.interieur.gouv.fr/telechargements/LG2002/T2Valide/029/circons05.xml
error https://www.archives-resultats-elections.interieur.gouv.fr/telechargements/LG2002/T2Valide/035/circons05.xml
error https://www.archives-resultats-elections.interieur.gouv.fr/telechargements

,Nom,Prenom,Civilite,LibNua,NumDepCand,Voix,RapVoixExp,CodCirc,NbVoix,RapportExprimes
0,GODIN,ANDRE,M.,Socialiste,2,19 817,"42,60",00101,19817,42.60
1,BERTRAND,J.MICHEL,M.,Union pour la majorité présidentielle,3,26 697,"57,40",00101,26697,57.40
0,GUICHON,LUCIEN,M.,Union pour la majorité présidentielle,6,25 811,"61,89",00102,25811,61.89
1,DRUT-GORJU,ELIANE,Mme,Socialiste,22,15 891,"38,11",00102,15891,38.11
0,BERTRAND,HUBERT,M.,Radical de gauche,20,16 094,"40,08",00103,16094,40.08
...,...,...,...,...,...,...,...,...,...,...
1,BRIAL,Victor,M.,Union pour la majorité présidentielle,2,3 774,"50,39",98601,3774,50.39
0,LAFLEUR,Jacques,M.,Union pour la majorité présidentielle,2,12 670,"55,74",98801,12670,55.74
1,LE ROUX,Didier,M.,Union pour la démocratie française,8,10 059,"44,26",98801,10059,44.26
0,FROGIER,Pierre,M.,Union pour la majorité présidentielle,3,15 719,"55,71",98802,15719,55.71


In [11]:
t2_resultats.to_csv('lg2002/t2_resultats.csv', index=False)

In [12]:
t1_inscrits = (
    pd.concat([
        pd
        .read_xml(
            f'{base}/{c["CodDpt3Car"]}/circons{c["CodCirLg"]}.xml',
            xpath='.//Tour[./NumTour = "1"]//Inscrit',
            encoding='latin1',
            dtype="object"
        )
        .assign(
            CodCirc = c['CodDpt3Car'] + c['CodCirLg'],
            Inscrits = lambda df: df.NbrIns.str.replace(' ','').astype(int),
        )
        
        for idx, c in tqdm(list(circonscriptions.iterrows()))
    ])
)

t1_inscrits

  0%|          | 0/577 [00:00<?, ?it/s]

,NbrIns,RapIns,CodCirc,Inscrits
0,80 534,"100,00",00101,80534
0,83 249,"100,00",00102,83249
0,77 413,"100,00",00103,77413
0,98 271,"100,00",00104,98271
0,70 129,"100,00",00201,70129
...,...,...,...,...
0,9 376,"100,00",98601,9376
0,84 235,"100,00",98701,84235
0,65 659,"100,00",98702,65659
0,60 779,"100,00",98801,60779


In [13]:
t1_inscrits.to_csv('lg2002/t1_inscrits.csv', index=False)